In [1]:
from google.colab import drive
drive.mount('/content/gdrive') # para trabajar con las carpetas en el drive


import pandas as pd       # data analysis and manipulation tool
import numpy as np


Mounted at /content/gdrive


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv")
df1=df[['neighbourhood_group',
'room_type',
'latitude',
'longitude',
'price',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'availability_365'
]].fillna(0)

In [ ]:
df1.neighbourhood_group.mode()

0    Manhattan
dtype: object

Q1: Manhattan

In [3]:
#new variable to use in the logistic regression 
df1.loc[df1['price'] >= 152, 'above_Average'] = 1
df1.loc[df1['price'] < 152, 'above_Average'] = 0

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
df_full_train , df_test =train_test_split(df1,test_size=0.2, random_state=42)

In [ ]:
len(df1), len(df_full_train)+len(df_test)

(48895, 48895)

In [6]:
df_train , df_val =train_test_split(df_full_train,test_size=0.25, random_state=42)

In [ ]:
len(df_train)+len(df_val), len(df_full_train)

(39116, 39116)

In [7]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [8]:
y_train = df_train.above_Average.values
y_val = df_val.above_Average.values
y_test = df_test.above_Average.values

In [9]:
df_val3=df_val.reset_index(drop=True)
df_test3=df_test.reset_index(drop=True)
df_train3=df_train.reset_index(drop=True)

y_train3=np.log1p(df_train3.price.values)
y_val3=np.log1p(df_val3.price.values)
y_test3=np.log1p(df_test3.price.values)

In [10]:
del df_train['price']
del df_val['price']
del df_test['price']

In [ ]:
df1.corr()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_Average
latitude,1.000000,0.084788,0.033939,0.024869,-0.015389,-0.018758,0.019517,-0.010983,0.056385
longitude,0.084788,1.000000,-0.150019,-0.062747,0.059094,0.138516,-0.114713,0.082731,-0.267001
price,0.033939,-0.150019,1.000000,0.042799,-0.047954,-0.050564,0.057472,0.081829,0.418150
minimum_nights,0.024869,-0.062747,0.042799,1.000000,-0.080116,-0.124905,0.127960,0.144303,0.034715
number_of_reviews,-0.015389,0.059094,-0.047954,-0.080116,1.000000,0.589407,-0.072376,0.172028,-0.051785
reviews_per_month,-0.018758,0.138516,-0.050564,-0.124905,0.589407,1.000000,-0.047312,0.163732,-0.056040
calculated_host_listings_count,0.019517,-0.114713,0.057472,0.127960,-0.072376,-0.047312,1.000000,0.225701,0.170075
availability_365,-0.010983,0.082731,0.081829,0.144303,0.172028,0.163732,0.225701,1.000000,0.104255
above_Average,0.056385,-0.267001,0.418150,0.034715,-0.051785,-0.056040,0.170075,0.104255,1.000000


Q2: reviews_per_month and number_of_reviews are the variables with the biggest correlation

In [ ]:
df_train

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_Average
0,Brooklyn,Entire home/apt,40.72760,-73.94495,3,29,0.70,13,50,0.0
1,Manhattan,Private room,40.70847,-74.00498,1,0,0.00,1,7,0.0
2,Bronx,Entire home/apt,40.83149,-73.92766,40,0,0.00,1,0,0.0
3,Brooklyn,Entire home/apt,40.66448,-73.99407,2,3,0.08,1,0,0.0
4,Manhattan,Private room,40.74118,-74.00012,1,48,1.80,2,67,0.0
...,...,...,...,...,...,...,...,...,...,...
29332,Brooklyn,Private room,40.71748,-73.95685,6,5,0.13,1,0,0.0
29333,Brooklyn,Private room,40.66397,-73.98538,1,7,0.17,2,0,0.0
29334,Manhattan,Private room,40.79994,-73.97001,1,1,0.64,1,88,1.0
29335,Brooklyn,Private room,40.69585,-73.96344,60,0,0.00,1,0,0.0


In [ ]:
#mutual information score for the 2 categorical variables in the training set

In [ ]:
from sklearn.metrics import mutual_info_score

In [ ]:
round(mutual_info_score(df_train.above_Average,df_train.neighbourhood_group),2)

0.05

In [ ]:
round(mutual_info_score(df_train.above_Average,df_train.room_type),2)

0.14

Q3. room type has a bigger score

In [13]:
#one-hot encoding
from sklearn.feature_extraction import DictVectorizer

In [ ]:
del df_train['above_Average']
del df_val['above_Average']
del df_test['above_Average']

In [ ]:
train_dicts = df_train.to_dict(orient="records")
dv=DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val.to_dict(orient="records")
#dv=DictVectorizer(sparse=False)
X_val = dv.transform(val_dicts)


array([[ 50.     ,  13.     ,  40.7276 , ...,   1.     ,   0.     ,
          0.     ],
       [  7.     ,   1.     ,  40.70847, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.83149, ...,   1.     ,   0.     ,
          0.     ],
       ...,
       [ 88.     ,   1.     ,  40.79994, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.69585, ...,   0.     ,   1.     ,
          0.     ],
       [281.     ,   2.     ,  40.64438, ...,   1.     ,   0.     ,
          0.     ]])

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
model.coef_[0].round(2)
model.intercept_[0]

-0.0026402001215476995

In [ ]:
model.predict(X_train)
model.predict_proba(X_train)

array([[0.62672509, 0.37327491],
       [0.89615208, 0.10384792],
       [0.82567537, 0.17432463],
       ...,
       [0.87785885, 0.12214115],
       [0.98335562, 0.01664438],
       [0.46505707, 0.53494293]])

In [ ]:
y_pred = model.predict(X_train)
y_pred

array([0., 0., 0., ..., 0., 0., 1.])

In [ ]:
import sklearn
sklearn.metrics.accuracy_score(y_train,y_pred)

0.7891399938644034

In [ ]:
y_pred = model.predict(X_val)
sklearn.metrics.accuracy_score(y_val,y_pred).round(2)

0.79

Q4.accuracy over validation is 0.79

In [ ]:
df_train.columns

Index(['neighbourhood_group', 'room_type', 'latitude', 'longitude',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [ ]:
accuracy =[]
columns_delete = ['neighbourhood_group', 'room_type', 'latitude', 'longitude',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365']
dv=DictVectorizer(sparse=False)
  
for col in columns_delete:
  df_train_1=df_train.drop(columns=col)
  df_val_1=df_val.drop(columns=col)

  train_dicts = df_train_1.to_dict(orient="records")
  X_train = dv.fit_transform(train_dicts)

  val_dicts = df_val_1.to_dict(orient="records")
  X_val = dv.transform(val_dicts)

  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)
  accuracy.append(sklearn.metrics.accuracy_score(y_val,y_pred).round(2))

accuracy

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


[0.75, 0.72, 0.79, 0.79, 0.79, 0.79, 0.79, 0.79, 0.78]

we obtain difference deleting 'neighbourhood_group', 'room_type' and 'availability_365'. The lowest is obtain by deleting room_type

Q5. number_of_reviews and reviews_per_month give difference 0

In [19]:
del df_train3["above_Average"]
del df_val3["above_Average"]
del df_test3["above_Average"]

In [20]:



#del df_train3["price"]
#del df_val3["price"]
#del df_test3["price"]


train_dicts3=df_train3.to_dict(orient='records')
dv3=DictVectorizer(sparse=False)
X_train3=dv3.fit_transform(train_dicts3)
val_dicts3 = df_val3.to_dict(orient="records")
X_val3=dv3.transform(val_dicts3)


In [18]:
df_train3

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_Average
0,Brooklyn,Entire home/apt,40.72760,-73.94495,3,29,0.70,13,50,0.0
1,Manhattan,Private room,40.70847,-74.00498,1,0,0.00,1,7,0.0
2,Bronx,Entire home/apt,40.83149,-73.92766,40,0,0.00,1,0,0.0
3,Brooklyn,Entire home/apt,40.66448,-73.99407,2,3,0.08,1,0,0.0
4,Manhattan,Private room,40.74118,-74.00012,1,48,1.80,2,67,0.0
...,...,...,...,...,...,...,...,...,...,...
29332,Brooklyn,Private room,40.71748,-73.95685,6,5,0.13,1,0,0.0
29333,Brooklyn,Private room,40.66397,-73.98538,1,7,0.17,2,0,0.0
29334,Manhattan,Private room,40.79994,-73.97001,1,1,0.64,1,88,1.0
29335,Brooklyn,Private room,40.69585,-73.96344,60,0,0.00,1,0,0.0


In [22]:
def rmse (y,y_pred):
    error=y-y_pred
    se=error**2
    mse=se.mean()
    return np.sqrt(mse)

from sklearn.linear_model import Ridge

for r in [0, 0.01, 0.1, 1, 10]:

    model3=Ridge(alpha=r, random_state=42)
    model3.fit(X_train3, y_train3)
    y_pred3_r=model3.predict(X_val3)
    rmse3= rmse(y_val3, y_pred3_r).round(3)
    print (r, rmse3)
    

0 0.497
0.01 0.497
0.1 0.497
1 0.497
10 0.498
